### PRN - 18030142043
### Indian Names NER Assignment
### Course - Deep Learning
### Name - Subodh
### Followed first approch using Spacy with custom tag and thinc.neural.Model


In [1]:
from nltk.tokenize import sent_tokenize
import pandas as pd
from GoogleNews import GoogleNews

#### Code to scrape data from google news.

googlenews = GoogleNews()
googlenews.search('Hrithik Roshan')
for l in range(151,175):
    googlenews.getpage(l)
news = googlenews.result()
news

desc = ''
for d in news:
    desc = desc + d["desc"] + d["title"]
desc

senlist = sent_tokenize(desc)

m = ''
for l in range(0,len(senlist)-1):
    m = m+senlist[l]
file = open("scraped.txt","a")
file.write(m)
file.close()

In [2]:
f = open("scraped.txt", "r")
m = f.read()

In [3]:
ms = m.split('.')

In [4]:
len(ms)

3508

In [5]:
ms[0:10]

['Hrithik Roshan and Katrina Kaif displayed terrific chemistry and dancing skills in the title track of their film Bang Bang',
 'Shot in a stylised manner\xa0',
 '',
 '',
 "Hrithik Roshan-Vaani Kapoor's New 'War' Song Gives Ultra-modern ",
 '',
 '',
 "Despite the innumerable ups and downs in his life, Hrithik Roshan's calm and composed demeanour in public remains unaffected",
 'The actor\xa0',
 '']

In [6]:
ms = list(filter(lambda x: x!= '', ms))
len(ms)

1398

In [7]:
ms[0:10]

['Hrithik Roshan and Katrina Kaif displayed terrific chemistry and dancing skills in the title track of their film Bang Bang',
 'Shot in a stylised manner\xa0',
 "Hrithik Roshan-Vaani Kapoor's New 'War' Song Gives Ultra-modern ",
 "Despite the innumerable ups and downs in his life, Hrithik Roshan's calm and composed demeanour in public remains unaffected",
 'The actor\xa0',
 'Hrithik Roshan has a sweet birthday wish for Vivek Oberoi: A friend ',
 'If you are a Hrithik Roshan or Tiger Shroff fan, then you might also feel the same way their diehard fans recently reacted after reading a\xa0',
 'Hrithik Roshan and Tiger Shroff are regular actors, jokes Arjun Kapoor ',
 'Hrithik Roshan shares a jaw dropping picture of himself in his latest post; ',
 ' Hrithik Roshan took to his Instagram account and shared a jow\xa0']

In [8]:
for l in range(0,len(ms)-1):
    ms[l] = ms[l].replace('Roshan','')
    ms[l] = ms[l].replace("Roshan's",'')
    ms[l] = ms[l].replace('\xa0','.')
    ms[l] = ms[l].replace(" \ ",'')
    ms[l] = ms[l].replace(" / ",' ')
    ms[l] = ms[l].lower()

In [9]:
len(ms)

1398

In [10]:
n  = pd.read_csv("indiannames.csv")
n.shape

(14845, 3)

In [11]:
n.dropna().any()
n.shape

(14845, 3)

In [12]:
names = n.name 
names = names.tolist()
for ll in range(0,len(names)):
    tmp = str(names[ll]).split(" ")
    names[ll] = tmp[0]
names.append("Hrithik")
print(len(names))

14846


In [13]:
train =[]
count = 0
for rl in range(0,len(ms)-1):
    if ("hrithik" in ms[rl]) == False:
        count = count + 1  
for rl in range(0,len(ms)-1-count):
    nrl = str(names[rl])
    if ("hrithik" in ms[rl]) == False:
        del ms[rl]
    else:
        ms[rl] = ms[rl].replace("hrithik",nrl,1)
        ms[rl] = ms[rl].replace("hrithik ",nrl,1)
        start_index = ms[rl].find(nrl)
        end_index = start_index + len(nrl)
        train.append((ms[rl],{'entities':[(start_index,end_index,'Indian Name')]}),)

In [14]:
len(train)

622

In [15]:
train[0:10]

[('barjraj  and katrina kaif displayed terrific chemistry and dancing skills in the title track of their film bang bang',
  {'entities': [(0, 7, 'Indian Name')]}),
 ("despite the innumerable ups and downs in his life, sharat 's calm and composed demeanour in public remains unaffected",
  {'entities': [(51, 57, 'Indian Name')]}),
 ('if you are a amit  or tiger shroff fan, then you might also feel the same way their diehard fans recently reacted after reading a.',
  {'entities': [(13, 17, 'Indian Name')]}),
 ('kushal  and tiger shroff are regular actors, jokes arjun kapoor ',
  {'entities': [(0, 6, 'Indian Name')]}),
 ('kasid  shares a jaw dropping picture of himself in his latest post; ',
  {'entities': [(0, 5, 'Indian Name')]}),
 (' shiv  took to his instagram account and shared a jow.',
  {'entities': [(1, 5, 'Indian Name')]}),
 ('vikram  shares a jaw dropping picture of himself in his latest ',
  {'entities': [(0, 6, 'Indian Name')]}),
 ("teachers' day 2019: from abhi  to aamir khan,

In [16]:
import spacy
import random
TRAIN_DATA = train[0:320]
def train_spacy(data,iterations):
    TRAIN_DATA = data
    nlp = spacy.blank('en')  
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)       
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']    
    with nlp.disable_pipes(*other_pipes):
        optimizer = nlp.begin_training()       
        for itn in range(iterations):
            print("Statring Iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}                        
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  
                    [annotations],  
                    drop=0.2,  
                    sgd=optimizer,
                    losses=losses)
            print(losses)
    return nlp
prdnlp = train_spacy(TRAIN_DATA,40)

Statring Iteration 0
{'ner': 249.8255034698731}
Statring Iteration 1
{'ner': 59.01628394641259}
Statring Iteration 2
{'ner': 45.80597654413178}
Statring Iteration 3
{'ner': 20.032914820548385}
Statring Iteration 4
{'ner': 27.26695585391709}
Statring Iteration 5
{'ner': 21.710609307004844}
Statring Iteration 6
{'ner': 17.32290066383283}
Statring Iteration 7
{'ner': 24.507257562663728}
Statring Iteration 8
{'ner': 12.32499549346111}
Statring Iteration 9
{'ner': 10.25974273684049}
Statring Iteration 10
{'ner': 17.586059209516293}
Statring Iteration 11
{'ner': 8.590581758342664}
Statring Iteration 12
{'ner': 16.04542150528008}
Statring Iteration 13
{'ner': 2.602922485336182}
Statring Iteration 14
{'ner': 0.0001831811567766912}
Statring Iteration 15
{'ner': 0.42143361300477966}
Statring Iteration 16
{'ner': 3.9996963296943377}
Statring Iteration 17
{'ner': 2.0292811556229795}
Statring Iteration 18
{'ner': 2.08101974835254}
Statring Iteration 19
{'ner': 18.728819301535513}
Statring Iteration

In [17]:
acc = 0
for i in range(320,400):
    test_text = ms[i]
    print("Test Statement \n",i-319)
    print(test_text)
    doc = prdnlp(test_text)
    for ent in doc.ents:
        print("Predicted Name : "+ent.text, 
              "\nLocation in sentence : ("+str(ent.start_char)+":" +str(ent.end_char)+")",
              "\nType : " +ent.label_)
        print("\n")
        if ent.text in names :
            acc = acc + 1
print("Accuracy of prediction :") 
print((acc/80)*100)

Test Statement 
 1
kangana ranaut and mohd  lock horns, yet againa source close to the development told the portal, "farah [khan] has already discussed the idea with mohd, who has also given a.
Predicted Name : mohd 
Location in sentence : (19:23) 
Type : Indian Name


Test Statement 
 2
rajmani  to play amitabh bachchan's role in farah khan's 
Predicted Name : rajmani 
Location in sentence : (0:7) 
Type : Indian Name


Test Statement 
 3
hrithik , who plays math.
Test Statement 
 4
mohan  drops super 30 new poster: vikas bahl gets director 
Predicted Name : mohan 
Location in sentence : (0:5) 
Type : Indian Name


Test Statement 
 5
super 30 box office collection: farid 's latest offering, super 30, based on the life of award-winning mathematician anand kumar is.
Predicted Name : farid 
Location in sentence : (32:37) 
Type : Indian Name


Test Statement 
 6
super 30 box office collection day 8: few takers for yuvraj  
Predicted Name : yuvraj 
Location in sentence : (53:59) 
Type : Ind

In [21]:
test = ["We are working on it from 2 months with rahul",
        " He has seen that movie with rajaram",
       ]
for i in range(0,2):
    test_text = test[i]
    print(test_text)
    doc = prdnlp(test_text)
    for ent in doc.ents:
        print("Predicted Name : "+ent.text, 
              "\nLocation in sentence : ("+str(ent.start_char)+":" +str(ent.end_char)+")",
              "\nType : " +ent.label_)

We are working on it from 2 months with rahul
Predicted Name : rahul 
Location in sentence : (40:45) 
Type : Indian Name
 He has seen that movie with rajaram


In [19]:
import numpy
nlp = spacy.blank('en')  
idv = nlp.vocab.strings[u"barjraj"]
nlp.vocab.vectors[idv]

KeyError: 11088361591375236284

In [ ]:
cat_id